# Capstone, Neighborhoods in Toronto

### The https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M was last edited on 30 march 2020.


#### Import libs.

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import lxml
from bs4 import BeautifulSoup

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


foli

#### Scraping wiki web page with beautifulSoup. and get table.

In [9]:
wiki_html_page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup_object = BeautifulSoup(wiki_html_page, "lxml")

# Table tag is: <table class="wikitable">
html_table = soup_object.find("table", class_="wikitable")

print(html_table.prettify())

<table class="wikitable">
 <tbody>
  <tr>
   <th>
    Postal code
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighborhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    North York
   </td>
   <td>
    Parkwoods
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    North York
   </td>
   <td>
    Victoria Village
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    Downtown Toronto
   </td>
   <td>
    Regent Park / Harbourfront
   </td>
  </tr>
  <tr>
   <td>
    M6A
   </td>
   <td>
    North York
   </td>
   <td>
    Lawrence Manor / Lawrence Heights
   </td>
  </tr>
  <tr>
   <td>
    M7A
   </td>
   <td>
    Downtown Toronto
   </td>
   <td>
    Queen's Park / Ontario Provincial Government
   </td>
  </tr>
  <tr>
   <td>
    M8A
   </td>
   <td>
    Not assigned
   </

#### Create Data Frame.

In [43]:
all_td = html_table.findAll("td")
print("Len of the all_td: ", len(all_td))

postalCode=[]
borough=[]
nieghberhood=[]

# create list of data for each column of data frame.
for data in range(0, 539, 3):
    
    postalCode.append(all_td[data].text)
    borough.append(all_td[data+1].text)
    nieghberhood.append(all_td[data+2].text)
    
print("len postalCode {}, borough {}, nie {}".format(len(postalCode), len(borough), len(nieghberhood)))


# create torento data frame.
df_torento_neighberhood = pd.DataFrame({"PostalCode": postalCode,
                                                          "Borough": borough,
                                                          "Neighberhood": nieghberhood
                                                         })

print("shape of the torento neighberhood Data Frame: ", df_torento_neighberhood.shape)
df_torento_neighberhood.head()

Len of the all_td:  540
len postalCode 180, borough 180, nie 180
shape of the torento neighberhood Data Frame:  (180, 3)


,PostalCode,Borough,Neighberhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n


#### Cleaning Data Of Torento Neighberhood Data Frame.

In [68]:
# in the https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M all data combined if needed,
# drop all rows have "Not assigned\n" value for column "Borough"
indexs = df_torento_neighberhood[ df_torento_neighberhood['Borough'] == "Not assigned\n" ].index

df_torento_neighberhood_clean_data = df_torento_neighberhood.drop(indexs)
df_torento_neighberhood_clean_data.reset_index(drop=True, inplace=True)

# strings have extra "\n" we replave is with ""
df_torento_neighberhood_clean_data["Neighberhood"].replace(to_replace=r'/', value=',', regex=True, inplace=True)
df_torento_neighberhood_clean_data["Neighberhood"].replace(to_replace=r'\n', value='', regex=True, inplace=True)
df_torento_neighberhood_clean_data["Borough"].replace(to_replace=r'\n', value='', regex=True, inplace=True)
df_torento_neighberhood_clean_data["PostalCode"].replace(to_replace=r'\n', value='', regex=True, inplace=True)

# check if we have null column
# print(df_torento_neighberhood_drop_no_assighned.isnull())

df_torento_neighberhood_clean_data

,PostalCode,Borough,Neighberhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Last cell print shape.

In [63]:
df_torento_neighberhood_clean_data.shape

(103, 3)